### Import Packages

In [33]:
import numpy as np
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
import pandas as pd
import mlflow
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.metrics import accuracy_score

### Load the Data

In [34]:
# Load the dataset
X,y = load_iris(return_X_y=True,as_frame=True)

X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [35]:
set_config(transform_output='pandas')

In [36]:
# train test split

random_state = 42
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size,random_state=random_state)

print('The shape of the train data is',X_train.shape) 
print('The shape of the test data is ',X_test.shape)

The shape of the train data is (120, 4)
The shape of the test data is  (30, 4)


In [37]:
scaler = StandardScaler()

label_encoder = LabelEncoder()

# fit_transform on the train data
X_train_trans = scaler.fit_transform(X_train)
y_train = label_encoder.fit_transform(y_train)

# transform on the test data
X_test_trans = scaler.transform(X_test)
y_test = label_encoder.transform(y_test)

In [38]:
X_train_trans

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
22,-1.473937,1.203658,-1.562535,-1.312603
15,-0.133071,2.992376,-1.276006,-1.045633
65,1.085898,0.085709,0.385858,0.289218
11,-1.230143,0.756479,-1.218701,-1.312603
42,-1.717731,0.309299,-1.390618,-1.312603
...,...,...,...,...
71,0.354517,-0.585060,0.156636,0.155733
106,-1.108246,-1.255829,0.443164,0.689673
14,-0.011174,2.098017,-1.447923,-1.312603
92,-0.011174,-1.032239,0.156636,0.022248


### MLFlow Tracking Code
. Hyperopt Search

> Start the mlflow server from the terminal using command (and let it run) : mlflow server --host 127.0.0.1 --port 8000

In [39]:
# define the search space
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 10),
    'max_depth': hp.quniform('max_depth', 5, 20, 1),
    'criterion': hp.choice('criterion',['gini','entropy'])
}

In [40]:
# define the objective function
def objective(params):
    with mlflow.start_run(nested=True) as parent:
        # log the current run parameters
        mlflow.log_params(params)
        # give the parameters as integers
        max_depth = int(params['max_depth'])
        n_estimators = int(params['n_estimators'])
        # train the model
        model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth)
        model.fit(X_train_trans,y_train)
        # get the accuracy score
        y_pred = model.predict(X_test_trans)
        # log the score
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric('accuracy_score',accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK}

In [41]:
best = fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=1000)

🏃 View run languid-hen-580 at: http://127.0.0.1:8000/#/experiments/105919964262218446/runs/7e9641a719384b52a1e0af5a383d9c68

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/105919964262218446

🏃 View run caring-jay-12 at: http://127.0.0.1:8000/#/experiments/105919964262218446/runs/1f1befc179f64d479b806fc221559633

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/105919964262218446

🏃 View run tasteful-goat-138 at: http://127.0.0.1:8000/#/experiments/105919964262218446/runs/a6e1ca2a98204ff28f774b1bc50ddc20

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/105919964262218446

🏃 View run chill-foal-785 at: http://127.0.0.1:8000/#/experiments/105919964262218446/runs/6344e8fa59e44a3c80d3bac7807e1a70

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/105919964262218446

🏃 View run trusting-wasp-518 at: http://127.0.0.1:8000/#/experiments/105919964262218446/runs/96e4dcc817844494a4d23b3c9d3a82b8

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/1059

In [42]:
best

{'criterion': np.int64(0),
 'max_depth': np.float64(20.0),
 'n_estimators': np.float64(110.0)}

In [52]:
# set the tracking uri

mlflow.set_tracking_uri("http://127.0.0.1:8000")

mlflow.set_experiment("HP Tuning Hyperopt v2")

with mlflow.start_run(run_name='Main',nested=True):
    # set the fmin function
    trials = Trials()
    
    best = fmin(fn=objective, 
                space=space, 
                algo=tpe.suggest, 
                max_evals=50, 
                trials=trials)

2025/09/03 00:13:37 INFO mlflow.tracking.fluent: Experiment with name 'HP Tuning Hyperopt v2' does not exist. Creating a new experiment.


🏃 View run efficient-deer-353 at: http://127.0.0.1:8000/#/experiments/181376187903889869/runs/70062fa96348480a97dede9e0c6c097c

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/181376187903889869

🏃 View run receptive-skunk-254 at: http://127.0.0.1:8000/#/experiments/181376187903889869/runs/74e805bfc8264c16ac2553be9c33bdc2

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/181376187903889869

🏃 View run tasteful-loon-283 at: http://127.0.0.1:8000/#/experiments/181376187903889869/runs/0aeaa3e00c3a40c4a9aafe7824b07088

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/181376187903889869

🏃 View run selective-fawn-238 at: http://127.0.0.1:8000/#/experiments/181376187903889869/runs/0315628d7da84902901bf50c84a83217

🧪 View experiment at: http://127.0.0.1:8000/#/experiments/181376187903889869

🏃 View run powerful-owl-353 at: http://127.0.0.1:8000/#/experiments/181376187903889869/runs/913e81f6ddaf47a2bb88582f6802b5f8

🧪 View experiment at: http://127.0.0.1:8000/#/expe